# Test Consistencia

In [56]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn

from copy import deepcopy
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm

from bert import BERT
from utils_vocab import BasicTokenizer, BERTDatasetNoLabels, evaluate

from sklearn.metrics import accuracy_score, f1_score

# You can also use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

In [57]:
tokenizer_file = 'tokenizer_5.pkl'
# tokenizer_file = 'tokenizer_10.pkl'
# tokenizer_file = 'tokenizer_15.pkl'
# tokenizer_file = 'tokenizer_20.pkl'

raw_dataset = 'implicacion_inversa_5.csv'
# raw_dataset = 'implicacion_inversa_10.csv'
# raw_dataset = 'implicacion_inversa_15.csv'
# raw_dataset = 'implicacion_inversa_20.csv'

path_model = 'implicacion_inversa_5.pt'
# path_model = 'implicacion_inversa_10.pt'
# path_model = 'implicacion_inversa_15.pt'
# path_model = 'implicacion_inversa_20.pt'

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(device)

mps


### Crear el Tokenizer

In [58]:
special_symbols = ['[UNK]', '[PAD]', '[CLS]', '[SEP]', '[MASK]']
PAD_IDX = 1
CLS_IDX = 2
SEP_IDX = 3

simple_tokenizer = lambda tokens_string: tokens_string.strip().split()
tokenizer = BasicTokenizer.create_using_stoi(simple_tokenizer, special_symbols, tokenizer_file)
print('vocabulary_size:', tokenizer.get_vocab_size())

vocabulary_size: 41


### Cargar datos y crear dataloader

In [59]:
df = pd.read_csv(raw_dataset)
print(df.shape)

(18899, 4)


In [60]:
def direct_prepare_bert_final_inputs(sentences1, sentences2, is_nexts, is_consistents, to_tensor=True):
    """
    Prepare the final input lists for BERT training.
    """
    def zero_pad_list_pair(pair_, pad=PAD_IDX):
        pair = deepcopy(pair_)
        max_len = max(len(pair[0]), len(pair[1]))
        #append [PAD] to each sentence in the pair till the maximum length reaches
        pair[0].extend([pad] * (max_len - len(pair[0])))
        pair[1].extend([pad] * (max_len - len(pair[1])))
        return pair[0], pair[1]

    #flatten the tensor
    flatten = lambda l: [item for sublist in l for item in sublist]

    bert_inputs_final, bert_labels_final, segment_labels_final, is_nexts_final, is_consistents_final = [], [], [], [], []

    for sentence1, sentence2, is_next, is_consistent in zip(sentences1, sentences2, is_nexts, is_consistents):
        # Tokenize each sentence
        tokens1 = tokenizer.encode(sentence1).ids
        tokens2 = tokenizer.encode(sentence2).ids
        bert_input = ([CLS_IDX] + tokens1 + [SEP_IDX], tokens2 + [SEP_IDX])

        # Create segment labels for each pair of sentences
        segment_label = [[1] * len(bert_input[0]), [2] * len(bert_input[1])]

        # Zero-pad the bert_input and bert_label and segment_label
        bert_input_padded = zero_pad_list_pair(bert_input)
        segment_label_padded = zero_pad_list_pair(segment_label,pad=0)

        bert_inputs_final.append(flatten(bert_input_padded))
        segment_labels_final.append(flatten(segment_label_padded))
        is_nexts_final.append(is_next)
        is_consistents_final.append(is_consistent)

    return bert_inputs_final, segment_labels_final, is_nexts_final, is_consistents_final

In [61]:
sentences1 = df.iloc[:, 0]
sentences2 = df.iloc[:, 1]
is_nexts = df.iloc[:, 2]
is_consistent = df.iloc[:, 3]
bert_inputs_final, segment_labels_final, is_nexts_final, is_consistent_final = direct_prepare_bert_final_inputs(sentences1, sentences2, is_nexts, is_consistent)

In [62]:
df_final = pd.DataFrame({
    'BERT Input': bert_inputs_final,
    'Segment Label': segment_labels_final,
    'Is Next': is_nexts_final,
    'Is Consistent': is_consistent_final
})
print(f'{df_final.shape=}')

df_final.shape=(18899, 4)


In [63]:
dataset = BERTDatasetNoLabels(df_final)
bert_inputs, segment_labels, is_nexts, is_consistents = dataset[1]

In [64]:
PAD_IDX = 1

def collate_batch(batch):
    bert_inputs_batch, segment_labels_batch, is_nexts_batch, is_consistents_batch = [], [], [], []

    for bert_input, segment_label, is_next, is_consistent in batch:
        # Convert each sequence to a tensor and append to the respective list
        bert_inputs_batch.append(torch.tensor(bert_input, dtype=torch.long))
        segment_labels_batch.append(torch.tensor(segment_label, dtype=torch.long))
        is_nexts_batch.append(torch.tensor(is_next, dtype=torch.long))
        is_consistents_batch.append(torch.tensor(is_consistent, dtype=torch.long))

    # Pad the sequences in the batch
    bert_inputs_final = pad_sequence(bert_inputs_batch, padding_value=PAD_IDX, batch_first=False)
    segment_labels_final = pad_sequence(segment_labels_batch, padding_value=PAD_IDX, batch_first=False)
    is_nexts_batch = torch.tensor(is_nexts_batch)
    is_consistents_batch = torch.tensor(is_consistents_batch)

    return bert_inputs_final.to(device), segment_labels_final.to(device), is_nexts_batch.to(device), is_consistents_batch.to(device)

In [65]:
BATCH_SIZE = 128

dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)


In [66]:
bert_inputs, segment_labels, is_nexts, is_consistents = next(iter(dataloader))

### Cargar el modelo

In [67]:
EMBEDDING_DIM = 16

# Define parameters
vocab_size = tokenizer.get_vocab_size()  # Replace VOCAB_SIZE with your vocabulary size
d_model = EMBEDDING_DIM  # Replace EMBEDDING_DIM with your embedding dimension
n_layers = 4  # Number of Transformer layers
initial_heads = 4
# Ensure the number of heads is a factor of the embedding dimension
heads = initial_heads - d_model % initial_heads

dropout = 0.1  # Dropout rate

# Create an instance of the BERT model
model = BERT(vocab_size, d_model, n_layers, heads, dropout)
model.to(device)

model.load_state_dict(torch.load(path_model, weights_only=True,map_location=torch.device(device)))
model.eval()

BERT(
  (bert_embedding): BERTEmbedding(
    (token_embedding): TokenEmbedding(
      (embedding): Embedding(41, 16)
    )
    (positional_encoding): PositionalEncoding(
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (segment_embedding): Embedding(3, 16)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder_layer): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=16, out_features=16, bias=True)
    )
    (linear1): Linear(in_features=16, out_features=32, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (linear2): Linear(in_features=32, out_features=16, bias=True)
    (norm1): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.1, inplace=False)
    (dropout2): Dropout(p=0.1, inplace=False)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-3): 4 x TransformerE

In [68]:
bert_inputs, segment_labels, is_nexts, is_consistents = next(iter(dataloader))

In [69]:
bert_inputs.cpu().numpy().transpose(1, 0).tolist()

[[2, 24, 18, 13, 21, 11, 3, 5, 13, 21, 24, 11, 3, 1, 1, 1, 1, 1, 1, 1],
 [2, 18, 14, 21, 7, 17, 11, 3, 1, 1, 24, 5, 14, 21, 24, 7, 16, 24, 11, 3],
 [2, 24, 18, 13, 19, 9, 3, 18, 13, 19, 9, 3, 1, 1, 1, 1, 1, 1, 1, 1],
 [2, 18, 12, 9, 17, 10, 3, 18, 12, 23, 9, 17, 10, 3, 1, 1, 1, 1, 1, 1],
 [2, 18, 13, 10, 17, 7, 3, 1, 5, 13, 24, 10, 16, 24, 7, 3, 1, 1, 1, 1],
 [2, 24, 5, 6, 9, 16, 11, 3, 18, 6, 24, 9, 17, 24, 11, 3, 1, 1, 1, 1],
 [2, 5, 14, 20, 7, 17, 10, 3, 1, 18, 14, 20, 24, 7, 16, 24, 10, 3, 1, 1],
 [2, 24, 18, 13, 20, 8, 3, 18, 13, 20, 8, 3, 1, 1, 1, 1, 1, 1, 1, 1],
 [2, 18, 13, 23, 11, 3, 5, 13, 23, 24, 11, 3, 1, 1, 1, 1, 1, 1, 1, 1],
 [2, 18, 12, 20, 24, 9, 16, 24, 8, 3, 25, 12, 20, 9, 17, 8, 3, 1, 1, 1],
 [2, 24, 18, 13, 19, 10, 3, 18, 13, 19, 10, 3, 1, 1, 1, 1, 1, 1, 1, 1],
 [2, 18, 15, 9, 16, 11, 3, 18, 15, 22, 9, 16, 11, 3, 1, 1, 1, 1, 1, 1],
 [2, 5, 14, 22, 11, 3, 18, 14, 22, 24, 11, 3, 1, 1, 1, 1, 1, 1, 1, 1],
 [2, 18, 13, 20, 24, 11, 16, 24, 9, 3, 25, 13, 20, 11, 17, 9, 3, 

In [70]:
with torch.no_grad():
    next_sentence_prediction, masked_language = model(bert_inputs.to(device), segment_labels.to(device))

In [71]:
def invert_sentences(bert_input):
    def zero_pad_list_pair(pair_, pad=PAD_IDX):
        pair = deepcopy(pair_)
        max_len = max(len(pair[0]), len(pair[1]))
        #append [PAD] to each sentence in the pair till the maximum length reaches
        pair[0].extend([pad] * (max_len - len(pair[0])))
        pair[1].extend([pad] * (max_len - len(pair[1])))
        return pair[0] + pair[1]

    tokens = bert_input.cpu().numpy().tolist()
    assert(len(tokens) % 2 == 0), len(tokens)
    try:
        len_sents = tokens.index(SEP_IDX)
    except Exception as e:
        print(tokens)
        print(tokenizer.decode(tokens))
        raise Exception(e)
    first = tokens[1:len_sents - 1]
    second_ = tokens[len_sents:]
    try:
        sep_in_second = second_.index(SEP_IDX)
    except Exception as e:
        print(second_)
        print(tokenizer.decode(second_))
        raise Exception(e)
    second = second_[:sep_in_second]
    bert_input = ([CLS_IDX] + second + [SEP_IDX], first + [SEP_IDX])
    # Create segment labels for each pair of sentences
    segment_label = ([1] * len(bert_input[0]), [2] * len(bert_input[1]))
    # Zero-pad the bert_input and bert_label and segment_label
    bert_input_padded = zero_pad_list_pair(bert_input)
    bert_input_padded = torch.tensor(bert_input_padded, dtype=torch.long).unsqueeze(dim=1)
    segment_label_padded = zero_pad_list_pair(segment_label,pad=0)
    segment_label_padded = torch.tensor(segment_label_padded, dtype=torch.long).unsqueeze(dim=1)
    return bert_input_padded, segment_label_padded

In [72]:
def get_prediction(input, segment_label, is_next, device):
    with torch.no_grad():
        model_prediction, _ = model(input.to(device), segment_label.to(device))
    logits = torch.softmax(model_prediction, dim=1)
    prediction = torch.argmax(logits, dim=1)     
    correct = prediction == is_next
    return correct, prediction

In [73]:
test = list()
golden = list()
for bert_inputs, segment_labels, is_nexts, is_consistents in tqdm(dataloader):
    corrects,_ = get_prediction(
        input=bert_inputs,
        segment_label=segment_labels,
        is_next=is_nexts,
        device=device
    )
    for idx, correct in enumerate(corrects):
        if correct:
            bert_input = bert_inputs.transpose(1,0)[idx]
            bert_input_reversed, segment_label = invert_sentences(bert_input)
            check, prediction = get_prediction(
                input=bert_input_reversed, 
                segment_label=segment_label,
                is_next=is_consistents[idx],
                device=device
            )
            test.append(prediction.cpu().item())
            golden.append(is_consistents[idx].cpu())
acc = accuracy_score(golden, test)
f1 = f1_score(golden, test)    

print(f"F1 Score: {f1}")
print(f"Accuracy: {acc}")


  0%|          | 0/148 [00:00<?, ?it/s]

F1 Score: 0.5916296234169253
Accuracy: 0.5660427807486631
